# pandas 라이브러리와 pymysql

In [22]:
import pymysql
import pandas as pd

In [ ]:
# 데이터베이스를 생성하는 SQL 쿼리
# student_mgmt 라는 데이터베이스가 없으면 새로 만든다
create_db_query = """
CREATE DATABASE IF NOT EXISTS student_mgmt;
"""

# 사용할 데이터베이스를 선택하는 SQL 쿼리
# 앞으로 실행할 SQL 명령들이 student_mgmt DB에서 실행됨
use_db_query = """
USE student_mgmt;
"""

# students 테이블을 삭제하는 SQL 쿼리
# 이미 students 테이블이 있으면 삭제하고, 없으면 그냥 넘어감
drop_table_query = """
DROP TABLE IF EXISTS students;
"""

# students 테이블을 새로 생성하는 SQL 쿼리
create_table_query = """
CREATE TABLE students (
id TINYINT NOT NULL AUTO_INCREMENT,
name VARCHAR(10) NOT NULL,
gender ENUM('man','woman') NOT NULL,
birth DATE NOT NULL,
english TINYINT NOT NULL,
math TINYINT NOT NULL,
korean TINYINT NOT NULL,
PRIMARY KEY (id)
) ENGINE=InnoDB DEFAULT CHARSET=utf8;
"""

In [ ]:
# students 테이블에 여러 명의 학생 데이터를 한 번에 넣는 SQL 쿼리
inserte_data_query = """
INSERT INTO students (name, gender, birth, english, math, korean) VALUES ('dave', 'man', '1983-07-16', 90, 80, 71);
INSERT INTO students (name, gender, birth, english, math, korean) VALUES ('minsun', 'woman', '1982-10-16', 30, 88, 60);
INSERT INTO students (name, gender, birth, english, math, korean) VALUES ('david', 'man', '1982-12-10', 78, 77, 30);
INSERT INTO students (name, gender, birth, english, math, korean) VALUES ('jade', 'man', '1979-11-1', 45, 66, 20);
INSERT INTO students (name, gender, birth, english, math, korean) VALUES ('jane', 'man', '1990-11-12', 65, 32, 90);
INSERT INTO students (name, gender, birth, english, math, korean) VALUES ('wage', 'woman', '1982-1-13', 76, 30, 80);
INSERT INTO students (name, gender, birth, english, math, korean) VALUES ('tina', 'woman', '1982-12-3', 87, 62, 71);
"""

In [5]:
# password에는 본인의 MySQL 비밀번호를 입력
host_name = 'localhost'
host_port = 3306
username = 'root'
password = 'sqlsql123123'
charset = 'utf8'

In [6]:
# db 연결
db = pymysql.connect(
 host=host_name, # MySQL Server Address
 port=host_port, # MySQL Server Port
 user=username, # MySQL username
 password=password, # password for MySQL username
 charset=charset
)

In [7]:
cursor = db.cursor()

In [ ]:
# 데이터베이스를 생성한다 (없으면 생성)
cursor.execute(create_db_query)

# 사용할 데이터베이스를 선택한다
cursor.execute(use_db_query)

# students 테이블이 이미 있으면 삭제한다
cursor.execute(drop_table_query)

# students 테이블을 새로 생성한다
cursor.execute(create_table_query)

# 지금까지 실행한 SQL 작업을 실제 DB에 저장(확정)한다
db.commit()

In [ ]:
# pymysql은 기본적으로 여러 개의 INSERT 문을 한 번에 실행하는 것을 허용하지 않는다
# 즉, execute() 함수는 ';'로 끝나는 SQL 문 1개만 실행할 수 있다
# 그래서 여러 개의 INSERT 문이 들어있는 문자열을
# ';' 기준으로 나눠서 하나씩 실행해야 한다

for query in inserte_data_query.split(";"):  # ';'를 기준으로 SQL 문을 나눈다
    query = query.strip()                     # 앞뒤 공백과 줄바꿈 제거
    if query:                                 # 빈 문자열이 아닐 때만 실행
        cursor.execute(query)                 # SQL 문 1개 실행

# 모든 INSERT 작업을 실제 DB에 저장(확정)
db.commit()

In [ ]:
# 실행할 SQL 문장을 문자열로 저장
# 현재 사용 중인 데이터베이스에 있는 모든 테이블 목록을 조회
sql = "SHOW TABLES"

In [ ]:
# SQL을 실행하고, 결과를 pandas DataFrame(df) 형태로 가져온다
df = pd.read_sql(sql, db)

C:\Users\User\AppData\Local\Temp\ipykernel_2704\3349604202.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, db)


In [12]:
df

,Tables_in_student_mgmt
0,students


In [ ]:
# students 테이블에 있는 모든 데이터 조회
sql = "SELECT * FROM students"

# 조회 결과를 DataFrame으로 가져온다
df = pd.read_sql(sql, db)

C:\Users\User\AppData\Local\Temp\ipykernel_2704\3088303204.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql,db)


In [14]:
df

,id,name,gender,birth,english,math,korean
0,1,dave,man,1983-07-16,90,80,71
1,2,minsun,woman,1982-10-16,30,88,60
2,3,david,man,1982-12-10,78,77,30
3,4,jade,man,1979-11-01,45,66,20
4,5,jane,man,1990-11-12,65,32,90
5,6,wage,woman,1982-01-13,76,30,80
6,7,tina,woman,1982-12-03,87,62,71


In [ ]:
# math 컬럼의 첫 번째 값의 자료형을 확인
# MySQL의 TINYINT 타입이 pandas에서는 정수형으로 잘 유지되는지 확인
type(df['math'][0])

numpy.int64

In [ ]:
# DataFrame(df)을 CSV 파일로 저장
# sep=','        : 컬럼 구분자를 콤마로 설정
# index=False    : DataFrame의 인덱스 번호는 파일에 저장하지 않음
# encoding='utf-8' : 한글이 깨지지 않도록 인코딩 설정
df.to_csv('students.csv', sep=',', index=False, encoding='utf-8')
df

,id,name,gender,birth,english,math,korean
0,1,dave,man,1983-07-16,90,80,71
1,2,minsun,woman,1982-10-16,30,88,60
2,3,david,man,1982-12-10,78,77,30
3,4,jade,man,1979-11-01,45,66,20
4,5,jane,man,1990-11-12,65,32,90
5,6,wage,woman,1982-01-13,76,30,80
6,7,tina,woman,1982-12-03,87,62,71


In [17]:
db.close()

# 외래키(FOREIGN KEY)를 만드는 이유

In [24]:
host_name = 'localhost'
host_port = 3306
username = 'root'
password = 'sqlsql123123'

# pymysql을 이용해 MySQL 서버에 연결
# host     : DB 서버 주소
# port     : DB 서버 포트 번호 (보통 3306)
# user     : MySQL 접속 계정
# password : 비밀번호
# charset  : 한글 처리를 위한 문자셋 설정
db = pymysql.connect(
 host=host_name,
 port=host_port,
 user=username,
 password=password,
 charset='utf8'
)

In [25]:
# SQL 문을 실행하기 위한 커서(cursor) 생성
cursor = db.cursor()

In [28]:
# 여러 개의 SQL 쿼리가 들어있는 sqlDB.sql 파일을 읽기
# with 문을 사용하면 파일을 자동으로 닫아줌
with open('sqlDB.sql', 'r', encoding='utf-8') as f:
    sql_file = f.read()      # 파일 내용을 문자열로 모두 읽어옴

# ';'를 기준으로 SQL 문들을 나눔
sql_querys = sql_file.split(";")

In [29]:
# 나눠진 SQL 쿼리를 하나씩 실행
for query in sql_querys:
    query = query.strip()    # 앞뒤 공백 및 줄바꿈 제거
    if query:                # 빈 문자열이 아닐 경우만 실행
        cursor.execute(query)

In [30]:
# 실행한 모든 SQL 작업을 실제 DB에 저장(확정)
db.commit()

In [31]:
# usertbl 테이블의 모든 데이터 조회
sql = "SELECT * FROM usertbl"

# 조회 결과를 pandas DataFrame 형태로 가져오기
df = pd.read_sql(sql, db)
df

C:\Users\User\AppData\Local\Temp\ipykernel_2704\3967591929.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, db)


,userID,name,birthYear,addr,mobile1,mobile2,height,mDate
0,BBK,바비킴,1973,서울,010,0000000,176,2013-05-05
1,EJW,은지원,1972,경북,011,8888888,174,2014-03-03
2,JKW,조관우,1965,경기,018,9999999,172,2010-10-10
3,JYP,조용필,1950,경기,011,4444444,166,2009-04-04
4,KBS,김범수,1979,경남,011,2222222,173,2012-04-04
5,KKH,김경호,1971,전남,019,3333333,177,2007-07-07
6,LJB,임재범,1963,서울,016,6666666,182,2009-09-09
7,LSG,이승기,1987,서울,011,1111111,182,2008-08-08
8,SSK,성시경,1979,서울,None,None,186,2013-12-12
9,YJS,윤종신,1969,경남,None,None,170,2005-05-05


In [32]:
# buytbl 테이블의 모든 데이터 조회
sql = "SELECT * FROM buytbl"

# 조회 결과를 pandas DataFrame 형태로 가져오기
df = pd.read_sql(sql, db)
df

C:\Users\User\AppData\Local\Temp\ipykernel_2704\1836262334.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, db)


,num,userID,prodName,groupName,price,amount
0,1,KBS,운동화,None,30,2
1,2,KBS,노트북,전자,1000,1
2,3,JYP,모니터,전자,200,1
3,4,BBK,모니터,전자,200,5
4,5,KBS,청바지,의류,50,3
5,6,BBK,메모리,전자,80,10
6,7,SSK,책,서적,15,5
7,8,EJW,책,서적,15,2
8,9,EJW,청바지,의류,50,1
9,10,BBK,운동화,None,30,2


# buyTbl에 데이터를 추가

In [ ]:
# 데이터베이스에 SQL 명령을 보내기 위한 커서 생성
cursor = db.cursor()

# buyTbl 테이블에 새로운 구매 정보를 추가하는 SQL 문
# userID   : 구매한 사용자 ID
# prodName : 상품 이름
# groupName: 상품 분류
# price    : 상품 가격
# amount   : 구매 수량
sql = "INSERT INTO buyTbl (userID, prodName, groupName, price, amount) VALUES('STJ', '운동화', '의류', 30, 2);"

# INSERT SQL 문을 실행하여 데이터 추가
cursor.execute(sql)

# 변경된 내용을 실제 데이터베이스에 저장(확정)
db.commit()


# 에러가 나면 정상
# userTbl 에 userID가 STJ인 데이터가 없기 때문에

IntegrityError: (1452, 'Cannot add or update a child row: a foreign key constraint fails (`sqldb`.`buytbl`, CONSTRAINT `buytbl_ibfk_1` FOREIGN KEY (`userID`) REFERENCES `usertbl` (`userID`))')

In [34]:
# 데이터베이스에 SQL 명령을 보내기 위한 커서 생성
cursor = db.cursor()

# buyTbl 테이블에 새로운 구매 정보 추가하는 SQL 문
# BBK 사용자가 운동화를 2개 구매한 기록
sql = "INSERT INTO buyTbl (userID, prodName, groupName, price, amount) VALUES('BBK', '운동화', '의류', 30, 2);"

# INSERT SQL 문 실행
cursor.execute(sql)

# 변경된 내용을 실제 데이터베이스에 저장(확정)
db.commit()

# 데이터베이스 연결 종료 (자원 정리)
db.close()

In [35]:
# 데이터베이스 연결을 쉽게 재사용하기 위한 함수 정의
# d_name : 접속할 데이터베이스 이름
def conn(d_name):
    import pymysql

    # MySQL 서버 접속 정보
    host_name = 'localhost'
    host_port = 3306
    username = 'root'
    password = 'sqlsql123123'
    database_name = d_name

    # MySQL 데이터베이스에 연결
    db = pymysql.connect(
        host=host_name,      # MySQL 서버 주소
        port=host_port,      # MySQL 서버 포트 번호
        user=username,       # MySQL 접속 계정
        password=password,   # 비밀번호
        database=database_name,  # 사용할 데이터베이스 이름
        charset='utf8'       # 한글 처리 설정
    )

    # 생성된 DB 연결 객체 반환
    return db

In [36]:
# conn 함수를 사용해 sqlDB 데이터베이스에 연결
db = conn('sqlDB')

In [37]:
# DB 연결을 위한 커서 생성
cursor = db.cursor()  # db.cursor()는 MySQL과 통신할 수 있는 객체 생성

# userTbl 테이블에 데이터 삽입
sql = "INSERT INTO userTbl VALUES('STJ', '서태지', 1975, '경기', '011', 'toortoor', 171, '2014-4-4');"
cursor.execute(sql)  # SQL 쿼리를 실행
db.commit()          # 데이터베이스에 실제로 반영 (저장)

In [38]:
# buyTbl 테이블에 데이터 삽입
sql = "INSERT INTO buyTbl (userID, prodName, groupName, price, amount) VALUES('STJ', '운동화', '의류', 30, 2);"
cursor.execute(sql)  # SQL 쿼리를 실행
db.commit()          # 데이터베이스에 실제로 반영 (저장)

In [ ]:
# userTbl 테이블에서 특정 데이터를 삭제하는 코드
sql = "DELETE FROM userTbl WHERE userID='STJ'"  # userID가 'STJ'인 행(row)을 삭제하는 SQL 문
cursor.execute(sql)  # SQL 문 실행
db.commit()          # 삭제한 내용을 DB에 실제로 반영 (저장)

# 에러나면 정상
# buyTbl 에 해당 userID를 참조하는 데이터가 있기 때문

IntegrityError: (1451, 'Cannot delete or update a parent row: a foreign key constraint fails (`sqldb`.`buytbl`, CONSTRAINT `buytbl_ibfk_1` FOREIGN KEY (`userID`) REFERENCES `usertbl` (`userID`))')

In [40]:
db.close()